In [82]:
import numpy as np
from numba import jit
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import pyvista as pv
from scipy import interpolate
import surface_normalize
import torch

In [8]:
yield_hist = np.array([[0.3, 0.3, 0.45,  1.5,  1.5,  0.6, 0.3, 0.08, 0.04, 0.02,  0, \
                        0.02, 0.04, 0.08, 0.3, 0.6, 1.5, 1.5, 0.45, 0.3, 0.3], \
                        [  0,  5,   10,   20,   30,   40,   50,   60,   70,   80, 90, \
                         100, 110, 120, 130, 140, 150, 160, 170, 175, 180]])
yield_hist[1] *= np.pi/180
yield_func = interpolate.interp1d(yield_hist[1], yield_hist[0], kind='quadratic')

surface_generator = surface_normalize.surface_normal(center_with_direction=np.array([[50,50,10]]),range3D=np.array([[0, 100, 0, 100, 0, 150]]), InOrOut=[1],celllength=1e-9,  yield_hist=yield_hist)

In [9]:
def ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, point_cloud):
    # 将光线方向标准化（单位向量）
    ray_direction = ray_direction / np.linalg.norm(ray_direction)

    # 计算从光线起点到每个点的向量
    point_vectors = point_cloud - ray_origin

    # 计算点向量在光线方向上的投影长度
    t = np.dot(point_vectors, ray_direction)

    # 计算光线上投影点的坐标
    projections = ray_origin + np.outer(t, ray_direction)

    # 计算当前点到投影点的距离
    distances = np.linalg.norm(point_cloud - projections, axis=1)

    indice = np.where(distances < 0.5)[0]
    # 如果没有点满足条件，则返回空结果
    if indice.size == 0:
        return None
    near_point = point_cloud[indice]

    distances_near_point = np.linalg.norm(ray_origin - near_point, axis=1)
    # 找到最小距离和最近点
    min_index = np.argmin(distances_near_point)
    closest_point = point_cloud[indice[min_index]]
    # min_distance = distance_in[min_index]

    return closest_point.astype(int)

In [3]:
def find_intersection_face(ray_origin, ray_direction, box_center, box_half_size):
    # 计算立方体的最小角和最大角
    min_corner = box_center - box_half_size
    max_corner = box_center + box_half_size

    # 初始化记录相交时间的数组和相交面的数组
    t_near = -np.inf
    t_far = np.inf
    hit_normal = np.array([0, 0, 0])

    # 遍历三个轴（x, y, z）
    for i in range(3):
        if ray_direction[i] != 0:
            # 计算 t1 和 t2
            t1 = (min_corner[i] - ray_origin[i]) / ray_direction[i]
            t2 = (max_corner[i] - ray_origin[i]) / ray_direction[i]
            
            # 交换 t1 和 t2，如果 t1 > t2
            if t1 > t2:
                t1, t2 = t2, t1
            
            # 更新 t_near 和 t_far
            if t1 > t_near:
                t_near = t1
                hit_normal = np.array([0, 0, 0])
                hit_normal[i] = -1 if ray_direction[i] > 0 else 1
            
            if t2 < t_far:
                t_far = t2

            # 检查是否存在相交
            if t_near > t_far or t_far < 0:
                return None, None  # 不相交

    # 确定哪个面被击中
    if np.array_equal(hit_normal, [-1, 0, 0]):
        return 0  #"Left"
    elif np.array_equal(hit_normal, [1, 0, 0]):
        return 1  #"Right"
    elif np.array_equal(hit_normal, [0, -1, 0]):
        return 2  #"Front"
    elif np.array_equal(hit_normal, [0, 1, 0]):
        return 3  #"Back"
    elif np.array_equal(hit_normal, [0, 0, -1]):
        return 4  #"Bottom"
    elif np.array_equal(hit_normal, [0, 0, 1]):
        return 5  #"Top"
    return None

In [176]:
# 定义反应表和初始的 film 元素
react_table = np.array([[[0.700, 0, 1], [0.300, 0, 1]],
                        [[0.200, -1, 0], [0.075, 0, -1]]])
film_elemnt = np.array([1, 0], dtype=float)
def reaction(gas, film):
    # 计算总长度
    total_length = int(film.sum())

    # 预分配反应类型和反应速率
    reaction_type = np.empty(total_length, dtype=int)
    reaction_rate = np.empty(total_length, dtype=float)

    # 填充反应类型和反应速率
    current_index = 0
    for i in range(film.shape[0]):
        length = int(film[i])
        reaction_type[current_index:current_index + length] = i
        reaction_rate[current_index:current_index + length] = react_table[gas, i, 0]
        current_index += length

    # 生成随机数列以确定反应发生的概率
    reaction_list = np.random.rand(total_length)

    # 筛选出发生反应的粒子
    reaction_indices = np.where(reaction_rate > reaction_list)[0]
    
    # 如果有反应发生，随机选择一个反应
    if reaction_indices.size > 0:
        reaction_choice = np.random.choice(reaction_indices, 1)
        reaction_particle = reaction_type[reaction_choice][0]
        # print(reaction_particle)
        # 更新 film 元素的状态
        filmAddtion = np.array(react_table[gas, reaction_particle, 1:], dtype=int)
        return filmAddtion, reaction_particle, np.sum(react_table[gas, reaction_particle, 1:]) # film, react_type, etchingDepo 
    else:
        return np.zeros_like(film), -1, 0

In [177]:
def pickParticles(film, inCell, outFlow):
    # print(film[inCell[0], inCell[1], inCell[2],6:][0])
    indices = np.repeat(np.arange(film[inCell[0], inCell[1], inCell[2],6:][0].size), (film[inCell[0], inCell[1], inCell[2],6:])[0])
    np.random.shuffle(indices)
    # 选择前 N 个，统计各类别的抽取数量
    unique, counts = np.unique(indices[:outFlow[0]], return_counts=True) # np.unique(indices[:N+1], return_counts=True)

    film[inCell[0], inCell[1], inCell[2], unique+6] -= counts
    return film, unique, counts

In [348]:
def enclose_overflow(film, i, j, k):
    i = i % film.shape[0]
    j = j % film.shape[1]
    k = k % film.shape[2]
    if np.sum(film[i-1, j, k, 6:]) > 30:
        if (np.sum(film[i-1-1, j, k, 6:]) > 0) & (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i-1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i-1, j, k+1, 6:]) > 0):
            inCell = np.array([i-1, j, k])
            overflow = np.sum(film[i-1, j, k, 6:]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i+1, j, k, 6:]) > 30:
        if (np.sum(film[i+1+1, j, k, 6:]) > 0) & (np.sum(film[i+1, j-1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0):
            inCell = np.array([i+1, j, k])
            overflow = np.sum(film[i+1, j, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j-1, k, 6:]) > 30:
        if (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i, j-1-1, k, 6:]) > 0) & (np.sum(film[i+1, j, k, 6:]) > 0) \
            & (np.sum(film[i, j-1, k-1, 6:]) > 0) & (np.sum(film[i, j-1, k+1, 6:]) > 0):
            inCell = np.array([i, j-1, k])
            overflow = np.sum(film[i, j-1, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j+1, k, 6:]) > 30:
        if (np.sum(film[i-1, j+1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) & (np.sum(film[i, j+1+1, k, 6:]) > 0) \
            & (np.sum(film[i, j+1, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0):
            inCell = np.array([i, j+1, k])
            overflow = np.sum(film[i, j+1, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j, k-1, 6:]) > 30:
        if (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k-1, 6:]) > 0) \
            & (np.sum(film[i, j, k-1-1, 6:]) > 0) & (np.sum(film[i, j-1, k, 6:]) > 0):
            inCell = np.array([i, j, k-1])
            overflow = np.sum(film[i, j, k-1]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j, k+1, 6:]) > 30:
        if (np.sum(film[i-1, j, k+1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0) \
            & (np.sum(film[i, j-1, k, 6:]) > 0) & (np.sum(film[i, j, k+1+1, 6:]) > 0):
            inCell = np.array([i, j, k+1])
            overflow = np.sum(film[i, j, k+1]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts

    # less than 30
    elif np.sum(film[i-1, j, k, 6:]) < 30:
        if (np.sum(film[i-1-1, j, k, 6:]) > 0) & (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i-1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i-1, j, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i-1, j, k, 6:])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i-1, j, k][unique+6] += counts
    elif np.sum(film[i+1, j, k, 6:]) < 30:
        if (np.sum(film[i+1+1, j, k, 6:]) > 0) & (np.sum(film[i+1, j-1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i+1, j, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i+1, j, k][unique+6] += counts
    elif np.sum(film[i, j-1, k, 6:]) < 30:
        if (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i, j-1-1, k, 6:]) > 0) & (np.sum(film[i+1, j, k, 6:]) > 0) \
            & (np.sum(film[i, j-1, k-1, 6:]) > 0) & (np.sum(film[i, j-1, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j-1, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j-1, k][unique+6] += counts
    elif np.sum(film[i, j+1, k, 6:]) < 30:
        if (np.sum(film[i-1, j+1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) & (np.sum(film[i, j+1+1, k, 6:]) > 0) \
            & (np.sum(film[i, j+1, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j+1, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j+1, k][unique+6] += counts
    elif np.sum(film[i, j, k-1, 6:]) < 30:
        if (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k-1, 6:]) > 0) \
            & (np.sum(film[i, j, k-1-1, 6:]) > 0) & (np.sum(film[i, j-1, k, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j, k-1])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k-1][unique+6] += counts
    elif np.sum(film[i, j, k+1, 6:]) < 30:
        if (np.sum(film[i-1, j, k+1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0) \
            & (np.sum(film[i, j-1, k, 6:]) > 0) & (np.sum(film[i, j, k+1+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j, k+1])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k+1][unique+6] += counts
    return film

In [434]:
def surface_advancement(film, type):
    N = int(30)
    sumfilm = np.sum(film[:, :, :, 6:], axis=-1, dtype=int)
    # print(sumfilm[:, :, 2])
    if type == 1:
        upperCell = np.where(sumfilm > 2*N)
        # print(upperCell)
        if upperCell[0].size == 0:
            return film, False # film, block change
        else:
            outFlowParticles = sumfilm[upperCell] - N
            flux = film[upperCell[0], upperCell[1], upperCell[2], :6][0]
            if np.sum(flux) == 0:
                print(flux)
                print(film[upperCell[0], upperCell[1], upperCell[2]])
                print('------flux------')
            flux_rate = np.zeros(flux.shape[0] + 1)
            flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
            flow_random = np.random.rand()
            mask = (flow_random > flux_rate[:-1]) & (flow_random <= flux_rate[1:])
            flow_choice = np.where(mask)[0]

            film, unique, counts = pickParticles(film, upperCell, outFlowParticles)
            print(unique)
            print('------unique-----')
            if flow_choice == 0:
                film[upperCell[0] - 1, upperCell[1], upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0] - 1, upperCell[1], upperCell[2])
                newCell = np.array([upperCell[0] - 1, upperCell[1], upperCell[2]])
            elif flow_choice == 1:
                film[upperCell[0] + 1, upperCell[1], upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0] + 1, upperCell[1], upperCell[2])
                newCell = np.array([upperCell[0] + 1, upperCell[1], upperCell[2]])
            elif flow_choice == 2:
                film[upperCell[0], upperCell[1] - 1, upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1] - 1, upperCell[2])
                newCell = np.array([upperCell[0], upperCell[1] - 1, upperCell[2]])
            elif flow_choice == 3:
                film[upperCell[0], upperCell[1] + 1, upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1] + 1, upperCell[2])
                newCell = np.array([upperCell[0], upperCell[1] + 1, upperCell[2]])
            elif flow_choice == 4:
                film[upperCell[0], upperCell[1], upperCell[2] - 1,unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1], upperCell[2] - 1)
                newCell = np.array([upperCell[0], upperCell[1], upperCell[2] - 1])
            elif flow_choice == 5:
                film[upperCell[0], upperCell[1], upperCell[2] + 1,unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1], upperCell[2] + 1)
                newCell = np.array([upperCell[0], upperCell[1], upperCell[2] + 1])
            else:
                print('error flow')
        # return film, newCell
            return film, True
    elif type == -1:
        lowerCell = np.where(np.logical_and(sumfilm > 0, sumfilm < N/5))
        # print(lowerCell)
        if lowerCell[0].size == 0:
            return film, False
        else:
            outFlowParticles = sumfilm[lowerCell]
            flux = film[lowerCell[0], lowerCell[1], lowerCell[2], :6][0]
            flux_min = np.min(flux)
            flux -= flux_min
            flux_rate = np.zeros(flux.shape[0] + 1)
            flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
            flow_random = np.random.rand()
            mask = (flow_random > flux_rate[:-1]) & (flow_random <= flux_rate[1:])
            flow_choice = np.where(mask)[0]
            counts = film[lowerCell[0], lowerCell[1], lowerCell[2], 6:][0]
            unique = np.arange(film.shape[3]-6)
            film[lowerCell[0], lowerCell[1], lowerCell[2],unique+6] -= counts
            print(unique)
            print('------unique-----')
            if flow_choice == 0:
                film[lowerCell[0] - 1, lowerCell[1], lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0] - 1, lowerCell[1], lowerCell[2])
            elif flow_choice == 1:
                film[lowerCell[0] + 1, lowerCell[1], lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0] + 1, lowerCell[1], lowerCell[2])
            elif flow_choice == 2:
                film[lowerCell[0], lowerCell[1] - 1, lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1] - 1, lowerCell[2])
            elif flow_choice == 3:
                film[lowerCell[0], lowerCell[1] + 1, lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1] + 1, lowerCell[2])
            elif flow_choice == 4:
                film[lowerCell[0], lowerCell[1], lowerCell[2] - 1,unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1], lowerCell[2] - 1)
            elif flow_choice == 5:
                film[lowerCell[0], lowerCell[1], lowerCell[2] + 1,unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1], lowerCell[2] + 1)
            else:
                print('error flow')
            # return film, lowerCell
            return film, True


In [435]:
def surface_addition(film, react_point, react, flux):
    film[react_point[0], react_point[1], react_point[2], flux] += react[2]
    film[react_point[0], react_point[1], react_point[2], 6:] += react[0]
    return film

In [436]:
# particle data structrue [x, y, z, vx, vy, vz, typeID]
box_half_size = 0.5

def modeling(particle, film, surface):
    close_point = ray_point_cloud_intersection_vectorized(particle[:3], particle[3:6], surface)
    flux = find_intersection_face(particle[:3], particle[3:6], close_point, box_half_size)
    print(film[close_point[0], close_point[1],close_point[2]])
    react = reaction(int(particle[-1]), film[close_point[0], close_point[1],close_point[2], 6:])
    print(react)
    film = surface_addition(film, close_point, react, flux)
    film, filmChange = surface_advancement(film, react[2])
    return film

In [437]:
def scanZ2(film): # fast scanZ
    film = np.sum(film[:,:,:,6:], axis=-1)
    film = torch.Tensor(film)
    xshape, yshape, zshape = film.shape
    
    # 初始化一个全零的表面稀疏张量
    surface_sparse = torch.zeros((xshape, yshape, zshape))
    
    # 获取当前平面与前后平面的布尔索引
    current_plane = film != 0


    # 获取周围邻居的布尔索引
    neighbors = torch.zeros_like(film, dtype=torch.bool)
    
    neighbors[1:, :, :] |= film[:-1, :, :] == 0  # 上面
    neighbors[:-1, :, :] |= film[1:, :, :] == 0  # 下面
    neighbors[:, 1:, :] |= film[:, :-1, :] == 0  # 左边
    neighbors[:, :-1, :] |= film[:, 1:, :] == 0  # 右边
    neighbors[:, :, 1:] |= film[:, :, :-1] == 0  # 前面
    neighbors[:, :, :-1] |= film[:, :, 1:] == 0  # 后面
    
    # 获取满足条件的索引
    condition = current_plane & neighbors
    
    # 更新表面稀疏张量
    surface_sparse[condition] = 1
    
    return surface_sparse.to_sparse()

In [438]:
film = np.zeros((10, 10, 5, 8), dtype=int)

film[:, :, :3, 6] = 30

particle = np.array([5.3, 5.2, 4.9, 0.1, -0.06, -1.5, 0])

scanFilm = scanZ2(film)
points = scanFilm.indices().T
surface = points.numpy()

# print(surface)

In [439]:
for i in range(80):
    testFilm = modeling(particle, film, surface)

[ 0  0  0  0  0  0 30  0]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  1 30  1]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  2 30  2]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  3 30  3]
(array([0, 1]), 1, 1.0)
[ 0  0  0  0  0  4 30  4]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  5 30  5]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  6 30  6]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  7 30  7]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  8 30  8]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0  9 30  9]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 10 30 10]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 11 30 11]
(array([0, 1]), 1, 1.0)
[ 0  0  0  0  0 12 30 12]
(array([0, 1]), 1, 1.0)
[ 0  0  0  0  0 13 30 13]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 14 30 14]
(array([0, 1]), 1, 1.0)
[ 0  0  0  0  0 15 30 15]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 16 30 16]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 17 30 17]
(array([0, 1]), 0, 1.0)
[ 0  0  0  0  0 18 30 18]
(array([0, 1]), 1, 1.0)
[ 0  0  0  0  0 19 30 19]
(array([0, 1]), 0, 1.0)


C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3300191146.py:18: RuntimeWarning: invalid value encountered in divide
  flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3300191146.py:26: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if flow_choice == 0:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3300191146.py:30: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_choice == 1:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3300191146.py:34: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_

In [414]:
testFilm = modeling(particle, film, surface)

[ 0  0  0  0  0  0 30  0]
(array([0, 1]), 0, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [391]:
for i in range(31):
    testFilm = modeling(particle, film, surface)

[ 0  0  0  0  0 33 14 18]
(array([0, 1]), 0, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 34 14 19]
(array([0, 1]), 1, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 35 14 20]
(array([0, 1]), 0, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 36 14 21]
(array([0, 1]), 1, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 37 14 22]
(array([0, 1]), 0, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 38 14 23]
(array([0, 1]), 0, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 39 14 24]
(array([0, 1]), 1, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 40 14 25]
(array([0, 1]), 1, 1.0)
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0  0  0  0  0 

C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:15: RuntimeWarning: invalid value encountered in divide
  flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if flow_choice == 0:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_choice == 1:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:31: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_

In [373]:
print(np.sum(testFilm[:, :, 1, 6:], axis=-1))
print(np.sum(testFilm[:, :, 2, 6:], axis=-1))
print(np.sum(testFilm[:, :, 3, 6:], axis=-1))


[[30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]]
[[30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 57 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]
 [30 30 30 30 30 30 30 30 30 30]]
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 31  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [287]:
a = np.where(testFilm[:, :, :, :6] != 0)
print(testFilm[a[0], a[1], a[2]])

[[  0   0   0   0   0 -25   5   0]]


In [407]:
film = np.zeros((20, 20, 20, 8), dtype=int)

film[:, :, :8, 6] = 30

N = int(2e4)
particle = np.zeros((N, 7))

position = np.random.rand(N,2)*10
velosityXY = np.random.rand(N,2)
velosityZ = np.random.rand(N)*-1

particle[:, :2] = position
particle[:,2] = 19
particle[:, 3:5] = velosityXY
particle[:, 5] = velosityZ
particle[:, -1] = 0

box_half_size = 0.5
scanFilm = scanZ2(film)
points = scanFilm.indices().T
surface = points.numpy()
for i in range(particle.shape[0]):
    print(particle[i][:3])
    close_point = ray_point_cloud_intersection_vectorized(particle[i][:3], particle[i][3:6], surface)
    print(close_point)
    if close_point is None or close_point.size == 0:
        continue
    else:
        flux = find_intersection_face(particle[i][:3], particle[i][3:6], close_point, box_half_size)
        # print(film[close_point[0], close_point[1],close_point[2]])
        react = reaction(int(particle[i][-1]), film[close_point[0], close_point[1],close_point[2], 6:])
        # print(react)
        film = surface_addition(film, close_point, react, flux)
        film, filmChange = surface_advancement(film, react[2])
        if filmChange is True:
            scanFilm = scanZ2(film)
            points = scanFilm.indices().T
            surface = points.numpy()   

[ 7.80960627  2.44562603 19.        ]
None
[ 5.76729629  3.41654295 19.        ]
[18 11  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 6.79096062  9.89674611 19.        ]
[11 19  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 5.06499839  3.11668697 19.        ]
None
[ 5.29797725  4.67490729 19.        ]
[13 15  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0.81314176  3.02490174 19.        ]
[2 6 7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 9.87465056  9.4999474  19.        ]
None
[ 1.76401457  1.73502839 19.        ]
None
[ 3.74939349  5.75200498 19.        ]
[ 4 18  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 4.41188914  7.91086679 19.        ]
None
[ 4.35882431  8.88728054 19.        ]
None
[ 2.01128036  9.15655602 19.        ]
None
[ 1.29618297  0.7524263  19.        ]
None
[ 7.50716158  0.54183595 19.        ]
None
[ 1.08

C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:15: RuntimeWarning: invalid value encountered in divide
  flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if flow_choice == 0:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_choice == 1:
C:\Users\URCS049\AppData\Local\Temp\ipykernel_34808\3473764811.py:31: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif flow_

[5 9 7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 8.31444075  2.78190123 19.        ]
None
[ 0.20701811  7.21556549 19.        ]
None
[ 1.33365421  1.04047    19.        ]
[12  3  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 0.79162417  7.14034726 19.        ]
None
[ 9.85962055  5.66718442 19.        ]
None
[ 5.92981487  0.69783084 19.        ]
None
[ 8.85863947  1.85045094 19.        ]
None
[ 4.50036954  7.14424998 19.        ]
None
[ 5.24275376  5.42835636 19.        ]
None
[ 2.18971579  5.20351987 19.        ]
None
[ 8.69434931  8.95817834 19.        ]
None
[ 2.82632129  9.5504378  19.        ]
None
[ 6.92669191  9.2765614  19.        ]
[10 14  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 1.3816611   8.73002843 19.        ]
None
[ 8.56410038  1.65146112 19.        ]
[12 13  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 8.36081325  0.23726855 19.        ]

IndexError: index 20 is out of bounds for axis 1 with size 20

In [408]:
print(np.sum(film[:, :, 7, 6:], axis=-1))
print(np.sum(film[:, :, 8, 6:], axis=-1))

[[30 30 30 30 30 31 30 32 31 30 30 30 30 30 32 31 30 31 30 30]
 [30 30 30 32 31 33 31 31 32 30 35 35 30 37 34 33 32 31 33 31]
 [30 30 31 33 33 32 35 33 44 38 37 41 37 33 38 36 38 34 37 36]
 [30 33 34 31 33 34 40 41 38 39 39 36 39 42 39 36 40 42 45 36]
 [31 32 30 34 41 34 34 41 46 42 39 42 46 48 44 46 38 50 41 40]
 [30 30 33 35 32 37 41 52 50 48 50 48 46 45 52 44 46 44 36 41]
 [30 33 38 39 39 41 43 40 46 45 50 57 52 59 52 45 50 52 47 43]
 [30 30 38 37 37 46 53 41 52 53 30 54 58 57 30 49 48 52 46 53]
 [32 33 35 43 38 40 51 46 51 53 30 30 48 30 31 30 30 53 59 46]
 [31 32 38 38 37 42 47 48 30 54 30 30 35 30 36 58 32 30 52 54]
 [31 35 38 36 46 44 44 53 30 59 30 30 30 42 38 30 30 57 56 51]
 [30 30 32 40 46 51 55 57 33 30 39 30 30 30 41 34 30 30 50 52]
 [30 34 40 43 39 44 50 59 57 30 38 59 30 30 30 60 56 53 52 55]
 [30 37 38 36 42 51 47 30 30 30 58 30 30 59 30 51 30 30 58 30]
 [30 31 37 39 43 40 49 49 53 56 30 30 59 57 30 33 55 55 53 53]
 [30 32 33 35 44 49 51 59 30 30 50 31 30 30 59 33 30 56

In [404]:
a = np.where(np.sum(film[:, :, 8, 6:], axis=-1) != 0)
print(a)
print(film[a[0], a[1], 8])

(array([13], dtype=int64), array([13], dtype=int64))
[[ 0  0  2  0  0  0 16 17]]


In [406]:
print(film[a[0], a[1]+1, 7])

[[ 5  0  1  0  0 12 30 18]]


In [405]:
print(film[a[0], a[1], 7])

[[ 7  0  7  0  0 17 14 16]]


In [392]:
# film = np.zeros((10, 10, 5, 8), dtype=int)

# film[:, :, :3, 6] = 30

# particle = np.array([5.3, 5.2, 4.9, 0.1, -0.06, -1.5, 1])

# scanFilm = scanZ2(film)
# points = scanFilm.indices().T
# surface = points.numpy()

# box_half_size = 0.5

def modeling(particle, film):
    box_half_size = 0.5
    scanFilm = scanZ2(film)
    points = scanFilm.indices().T
    surface = points.numpy()
    for i in range(particle.shape[0]):
        print(particle[i][:3])
        close_point = ray_point_cloud_intersection_vectorized(particle[i][:3], particle[i][3:6], surface)
        print(close_point)
        if close_point is None or close_point.size == 0:
            continue
        else:
            flux = find_intersection_face(particle[i][:3], particle[i][3:6], close_point, box_half_size)
            # print(film[close_point[0], close_point[1],close_point[2]])
            react = reaction(int(particle[i][-1]), film[close_point[0], close_point[1],close_point[2], 6:])
            # print(react)
            film = surface_addition(film, close_point, react, flux)
            film, filmChange = surface_advancement(film, react[2])
            if filmChange is True:
                scanFilm = scanZ2(film)
                points = scanFilm.indices().T
                surface = points.numpy()         
    return film

In [395]:
film = np.zeros((20, 20, 20, 8), dtype=int)

film[:, :, :8, 6] = 30

N = int(1e4)
particle = np.zeros((N, 7))

position = np.random.rand(N,2)*10
velosityXY = np.random.rand(N,2)
velosityZ = np.random.rand(N)*-1

particle[:, :2] = position
particle[:,2] = 19
particle[:, 3:5] = velosityXY
particle[:, 5] = velosityZ
particle[:, -1] = 0

film = modeling(particle, film)

[ 4.45442373  6.58445607 19.        ]
None
[ 8.95682775  4.06388527 19.        ]
None
[ 8.70236461  0.97903304 19.        ]
None
[ 0.82744534  9.07815929 19.        ]
None
[ 3.44881431  5.87489411 19.        ]
None
[ 0.35909298  5.35133618 19.        ]
None
[ 7.11033701  6.22884635 19.        ]
[ 8 18  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 1.59892898  6.40273405 19.        ]
[ 3 17  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 6.97835406  5.12812275 19.        ]
None
[ 5.15841231  6.60375976 19.        ]
None
[ 4.28519924  4.9178652  19.        ]
None
[ 5.09812593  9.52386492 19.        ]
None
[ 1.03703056  8.23328819 19.        ]
None
[ 5.20588845  5.64879256 19.        ]
None
[ 7.03561603  7.27877332 19.        ]
[15 16  7]
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
[ 6.11001654  9.74451145 19.        ]
None
[ 6.07644816  1.7431026  19.        ]
[11 12  7]
(array([], dtype=int64), a

In [396]:
print(np.sum(film[:, :, 7, 6:], axis=-1))
print(np.sum(film[:, :, 8, 6:], axis=-1))

[[30 30 30 30 30 30 30 30 30 31 30 30 30 30 31 30 30 30 30 30]
 [30 30 30 30 31 30 32 33 31 31 32 32 31 34 32 33 32 34 30 31]
 [30 30 32 33 32 32 32 32 33 36 32 36 35 36 34 37 33 33 32 37]
 [30 31 31 30 30 33 33 33 36 32 35 38 38 34 33 36 44 35 33 35]
 [30 32 31 32 34 34 37 40 36 33 42 40 42 37 39 33 39 37 36 34]
 [30 32 31 34 35 35 37 35 34 34 38 40 43 32 43 41 44 37 35 37]
 [30 31 34 32 36 34 36 37 36 36 39 40 38 47 43 37 39 42 44 42]
 [30 32 31 37 38 36 42 39 41 39 44 43 44 45 43 42 45 46 36 37]
 [31 31 33 33 39 35 37 47 44 45 50 46 42 55 43 43 47 45 41 44]
 [30 30 35 35 36 40 38 40 45 47 42 52 55 49 50 53 47 48 48 40]
 [30 32 35 40 37 37 38 49 45 57 53 45 47 51 46 46 52 47 49 45]
 [31 32 34 35 37 37 46 52 42 48 47 45 51 55 48 53 49 50 45 40]
 [30 33 33 32 39 40 40 41 47 44 57 30 40 54 56 56 46 46 52 49]
 [30 33 32 38 43 38 42 46 49 52 48 56 46 54 51 52 45 48 45 43]
 [30 30 34 41 39 42 42 42 46 57 53 53 48 54 52 51 46 50 40 45]
 [30 31 31 37 36 39 44 43 41 45 55 46 48 49 45 43 48 45

In [79]:
scanFilm = scanZ2(film)
points = scanFilm.indices().T
pointsNP = points.numpy()
print(pointsNP)

[[0 0 2]
 [0 1 2]
 [0 2 2]
 [0 3 2]
 [0 4 2]
 [0 5 2]
 [0 6 2]
 [0 7 2]
 [0 8 2]
 [0 9 2]
 [1 0 2]
 [1 1 2]
 [1 2 2]
 [1 3 2]
 [1 4 2]
 [1 5 2]
 [1 6 2]
 [1 7 2]
 [1 8 2]
 [1 9 2]
 [2 0 2]
 [2 1 2]
 [2 2 2]
 [2 3 2]
 [2 4 2]
 [2 5 2]
 [2 6 2]
 [2 7 2]
 [2 8 2]
 [2 9 2]
 [3 0 2]
 [3 1 2]
 [3 2 2]
 [3 3 2]
 [3 4 2]
 [3 5 2]
 [3 6 2]
 [3 7 2]
 [3 8 2]
 [3 9 2]
 [4 0 2]
 [4 1 2]
 [4 2 2]
 [4 3 2]
 [4 4 2]
 [4 5 2]
 [4 6 2]
 [4 7 2]
 [4 8 2]
 [4 9 2]
 [5 0 2]
 [5 1 2]
 [5 2 2]
 [5 3 2]
 [5 4 2]
 [5 5 2]
 [5 6 2]
 [5 7 2]
 [5 8 2]
 [5 9 2]
 [6 0 2]
 [6 1 2]
 [6 2 2]
 [6 3 2]
 [6 4 2]
 [6 5 2]
 [6 6 2]
 [6 7 2]
 [6 8 2]
 [6 9 2]
 [7 0 2]
 [7 1 2]
 [7 2 2]
 [7 3 2]
 [7 4 2]
 [7 5 2]
 [7 6 2]
 [7 7 2]
 [7 8 2]
 [7 9 2]
 [8 0 2]
 [8 1 2]
 [8 2 2]
 [8 3 2]
 [8 4 2]
 [8 5 2]
 [8 6 2]
 [8 7 2]
 [8 8 2]
 [8 9 2]
 [9 0 2]
 [9 1 2]
 [9 2 2]
 [9 3 2]
 [9 4 2]
 [9 5 2]
 [9 6 2]
 [9 7 2]
 [9 8 2]
 [9 9 2]]


In [80]:
print(film[pointsNP[0][0], pointsNP[0][1], pointsNP[0][2]])

[ 0.  0.  0.  0.  0.  0.  0. 30.]


In [81]:
point_cloud = pv.PolyData(pointsNP)

# arrows = point_cloud.glyph(
#     orient='vectors',
#     scale=1000,
#     factor=2,
# )

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:57333/index.html?ui=P_0x20200249410_8&reconnect=auto" class="pyvis…